In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
file_list = drive.ListFile({'q': "'16m7knepIB8xAqLlo1mTvbqeEWGyrtsaj' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

  
train_downloaded = drive.CreateFile({'id': '18eFL3pb6HwYLUdz8cyMU0SICMpeqA1Xr'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '1fMD1Bnn4xSFrRf345_rq1Phd38VlYoKs'})
test_downloaded.GetContentFile('test.csv') 
sample_downloaded = drive.CreateFile({'id': '10WP5Nj83aiD2ZFs3ZKFIOAIsE651_Rml'})
sample_downloaded.GetContentFile('sample_submission.csv') 

title: Untitled0.ipynb, id: 1vjCEG2ZJpAMnPDIt1oNaQU_xSdA-xa05
title: images.csv, id: 1CZwVwidus6-MN20tp0ZFPRLZzpiNvT5c
title: label.csv, id: 181SOoxDqQKjWzyhT3lIpXLBW0GZl0GFo
title: img_id.csv, id: 1OwrfT4z-yIg2J_AKEkQGxava3ON3IpEi
title: sample_submission.csv, id: 10WP5Nj83aiD2ZFs3ZKFIOAIsE651_Rml
title: train.csv, id: 18eFL3pb6HwYLUdz8cyMU0SICMpeqA1Xr
title: test.csv, id: 1fMD1Bnn4xSFrRf345_rq1Phd38VlYoKs


# **IMPORT DEPENDENCIES**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split

In [0]:
# reading datasets, can use mnist dataset inbuilt in keras

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
# converting pandas dataframe to numpy arrays

Y = train['label'].values
X = train.drop('label',axis=1).values


In [26]:
# Normalising image pixels between 0 to 1

X = X/255
print(Y.shape)
print(X.shape)

(42000,)
(42000, 784)


In [28]:
# reshaping the image 

X = X.reshape(X.shape[0],28,28,1)
Y.shape


(42000,)

In [0]:
# One hot encoding 

Y = keras.utils.to_categorical(Y)

In [0]:
# Splitting data into training and testing data

train_X,test_X,train_Y,test_Y = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# **BUILDING THE MODEL**

In [0]:
# Model building

def make_model():
  model = keras.Sequential()
  
  #convolutional layer with rectified linear unit activation
  model.add(keras.layers.Conv2D(32, (3,3), input_shape = (28,28,1), activation = 'relu' ) ) #32 filters each of size (3,3)

  model.add(keras.layers.Conv2D(64, (3,3), activation = 'relu' ) ) #64 filters each of size (3,3)
  
  #pooling layer 
  model.add(keras.layers.MaxPooling2D( pool_size = (2,2) ) )
  
  # randomly switching off and on neurons to improve convergence
  model.add(keras.layers.Dropout(0.33))
  
  # fully connected neural network
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(128, activation='relu'))
  
  model.add(keras.layers.Dropout(0.5))
  
  # converting matrix into probabalities using softmax activation
  model.add(keras.layers.Dense(10, activation='softmax'))
  
  model.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )
  
  return model


# **TRAINING AND EVALUATING MODEL**

In [57]:
model = make_model()

log = model.fit(train_X, train_Y, batch_size=200, epochs = 25, verbose=1, validation_data = (test_X,test_Y)) # training the model


Train on 33600 samples, validate on 8400 samples
Epoch 1/25
33600/33600 [==============================] - 6s 179us/step - loss: 0.3983 - acc: 0.8768 - val_loss: 0.1014 - val_acc: 0.9712
Epoch 2/25
33600/33600 [==============================] - 5s 150us/step - loss: 0.1206 - acc: 0.9640 - val_loss: 0.0608 - val_acc: 0.9815
Epoch 3/25
33600/33600 [==============================] - 5s 150us/step - loss: 0.0862 - acc: 0.9742 - val_loss: 0.0533 - val_acc: 0.9839
Epoch 4/25
33600/33600 [==============================] - 5s 149us/step - loss: 0.0705 - acc: 0.9789 - val_loss: 0.0469 - val_acc: 0.9861
Epoch 5/25
33600/33600 [==============================] - 5s 148us/step - loss: 0.0626 - acc: 0.9805 - val_loss: 0.0388 - val_acc: 0.9875
Epoch 6/25
33600/33600 [==============================] - 5s 147us/step - loss: 0.0520 - acc: 0.9848 - val_loss: 0.0402 - val_acc: 0.9873
Epoch 7/25
33600/33600 [==============================] - 5s 147us/step - loss: 0.0468 - acc: 0.9848 - val_loss: 0.0363 - v

In [58]:
scores = model.evaluate(test_X, test_Y, verbose=1) # evaluating the model

print("\nAccuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

8400/8400 [==============================] - 1s 145us/step

Accuracy: 0.9913095238095239 
 Error: 0.8690476190476204
